In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
md = pd.read_csv('/kaggle/input/2020ml-mid/mem_data.csv')
mt = pd.read_csv('/kaggle/input/2020ml-mid/mem_transaction.csv')

In [ ]:
md.info()
md.isnull().sum()

### sms데이터 정수 처리

In [ ]:
md.SMS.value_counts()

In [ ]:
md.SMS = (md.SMS=='Y').astype(int)
md.SMS.value_counts()

In [ ]:
md.info()

In [ ]:
## 양/음력 정수 처리

In [ ]:
md.BIRTH_SL = (md.BIRTH_SL=='S').astype(int)
md.BIRTH_SL.value_counts()

## 구매 합계 로그 처리

In [ ]:
md.SALES_AMT.value_counts()

In [ ]:
f = md.SALES_AMT.where(md.SALES_AMT>=0, other=0)
f = np.log(f+1)
md.SALES_AMT = f

## 최근 방문 일자로부터 경과일 구하기

In [ ]:
f = pd.to_datetime(md.LAST_VST_DT)
f = (pd.to_datetime('2007-12-31') - f).dt.days
md['E_DAY'] = f
md.E_DAY.head()

## 등록일로부터 경과일 구하기

In [ ]:
f = pd.to_datetime(md.RGST_DT)
f = (pd.to_datetime('2007-12-31') - f).dt.days
md['R_DAY'] = f
md.R_DAY.head()

## 우편번호(ZIP_CD)에서 광역행정구역 데이터 가져오기


In [ ]:
f = [x[0] for x in md.ZIP_CD]
md['R_REGION'] = f
md.R_REGION = md.R_REGION.where(md.R_REGION != '-', other=0).astype(int)
md.R_REGION.head()

In [ ]:
## 우편번호(ZIP_CD)에서 광역행정구역과 구를 합쳐서 데이터 가져오기

In [ ]:
f = [x[0:2] for x in md.ZIP_CD]
md['R_DET'] = f
md.R_DET = md.R_DET.where(md.R_DET != '-', other=0).astype(int)
md.R_DET.head()

In [ ]:
md.head()

## 평균 구매액 구하기

In [ ]:
f = mt.groupby('MEM_ID')['SELL_AMT'].agg('mean').reset_index()
f.columns = ['MEM_ID','SELL_AMT']
md = md.merge(f, how='left')
md.iloc[:,-1] = md.iloc[:,-1].fillna(0)

## 최대 구매액 구하기

In [ ]:
f = mt.groupby('MEM_ID')['SELL_AMT'].agg('max').reset_index()
f.columns = ['MEM_ID','SELL_MAX']
md = md.merge(f, how='left')
md.iloc[:,-1] = md.iloc[:,-1].fillna(0)

In [ ]:
md.head()

### 포인트 적립 횟수 구하기

In [ ]:
f = mt[mt.MEMP_TP=='A'].groupby('MEM_ID')['SELL_AMT'].agg('size').reset_index()
f.columns = ['MEM_ID','POINT_N']
md = md.merge(f, how='left')
md.iloc[:,-1] = md.iloc[:,-1].fillna(0)

In [ ]:
md.head()

### 날짜 컬럼 추가

In [ ]:
md['month'] = md.LAST_VST_DT.agg(lambda x : pd.to_datetime(x).month)

#md['day'] = md.LAST_VST_DT.agg(lambda x : pd.to_datetime(x).day)

#md['date'] = md.LAST_VST_DT.agg(lambda x: ('월', '화', '수', '목', '금', '토', '일')[pd.to_datetime(x).weekday()])
md['date'] = md.LAST_VST_DT.agg(lambda x: pd.to_datetime(x).weekday())

#md['season']
#md['month_when']

In [ ]:
#md.drop(['ZIP_CD','LAST_VST_DT'],axis='columns',inplace=True)

d_col = ['BIRTH_SL','M_STORE_ID','ZIP_CD','RGST_DT','LAST_VST_DT']
md = md.drop(d_col, axis=1)
md.info()

In [ ]:
md.drop(['USED_PNT'],axis=1)

In [ ]:
# 이상치 제거
md.iloc[:,2:] = md.iloc[:,2:].apply(lambda x : x.clip(x.quantile(.05), x.quantile(.95)), axis = 0)

In [ ]:
from sklearn.impute import SimpleImputer

imputer = SimpleImputer(strategy="most_frequent")
md = pd.DataFrame(imputer.fit_transform(md), columns=md.columns)
md

In [ ]:
#md.iloc[:,2:] = np.log1p(md.iloc[:,2:])

In [ ]:
md.info()

In [ ]:
name = 'data_preparation_20201026_4.csv'
output_name = 'pred_20201026_4'

In [ ]:
md.to_csv(name, index=False)

In [ ]:
input_data = name
output_data = output_name

In [ ]:
%run /kaggle/input/2020mlmid/modeling.py

In [ ]:
modeling(input_data, output_data)

In [ ]:
md.info()